In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

这本笔记本是在[Ivan Nardini](https://github.com/inardini)的帮助下编写的。

在GCP上进行端到端机器学习：MLOps阶段2：实验：开始使用Vertex AI实验

在GitHub上查看：

在Colab上运行：

在Vertex AI Workbench中打开：

## 概述

本教程演示了如何在 Google Cloud 上将 Vertex AI 用于端到端的 MLOps 生产环境。本教程涵盖阶段 2：实验：开始使用 Vertex AI 实验。

### 目标

在本教程中，您将学习如何在使用`Vertex AI`进行训练时使用`Vertex AI Experiments`。

本教程使用以下 Google Cloud ML 服务：

- `Vertex AI Experiments`
- `Vertex ML Metadata`
- `Vertex AI Training`

执行的步骤包括：

- 本地（笔记本）训练
    - 创建一个实验
    - 在实验中创建第一个运行
    - 记录参数和度量
    - 创建艺术品谱系
    - 可视化实验结果
    - 执行第二次运行
    - 在实验中比较两次运行
- 云端（`Vertex AI`）训练
    - 在训练脚本中：
        - 创建一个实验
        - 记录参数和度量
        - 创建艺术品谱系
    - 创建一个`Vertex AI Training`自定义作业
    - 执行自定义作业
    - 可视化实验结果

### 推荐

在 Google Cloud 上进行端到端 MLOps 时，以下是一些在实验或正式训练模型时记录数据的最佳实践。

#### Python 日志

在本地进行临时培训时，请使用Python的日志包。

#### 云日志

在云端进行训练时，请使用`Google Cloud Logging`。

#### 实验

在执行实验以比较不同实验配置的结果时，请结合使用 Vertex AI 实验和日志记录。

### 数据集

本教程不使用数据集。有关示例数据集的引用仅用于演示目的。

成本
本教程使用Google Cloud的计费组件：

- Vertex AI

了解[Vertex AI定价](https://cloud.google.com/vertex-ai/pricing)，并使用[Pricing Calculator](https://cloud.google.com/products/calculator/)根据您的预期使用量生成成本估算。

## 安装

安装以下包以执行此笔记本。

In [ ]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME") and not os.getenv("VIRTUAL_ENV")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

! pip3 install --upgrade google-cloud-aiplatform {USER_FLAG} -q
! pip3 install {USER_FLAG} --upgrade gsutil -q

重新启动内核

安装了额外的包之后，您需要重新启动笔记本内核，让它可以找到这些包。

In [ ]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

##开始之前

###设置您的Google Cloud项目

**无论您的笔记本环境如何，以下步骤都是必需的。**

1. [选择或创建一个Google Cloud项目](https://console.cloud.google.com/cloud-resource-manager)。当您首次创建账户时，您会获得$300的免费信用额用于计算/存储成本。

1. [确保您的项目已启用计费](https://cloud.google.com/billing/docs/how-to/modify-project)。

1. [启用Vertex AI、Compute Engine、Cloud Storage和Cloud Logging APIs](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component,storage_component,logging)。

1. 如果您在本地运行此笔记本，您需要安装[Cloud SDK](https://cloud.google.com/sdk)。

1. 在下面的单元格中输入您的项目ID。然后运行该单元格，确保Cloud SDK对本笔记本中的所有命令使用正确的项目。

**注意**：Jupyter会将以`!`为前缀的行视为shell命令，并将以`$`为前缀的Python变量插入这些命令中。

设置您的项目ID

**如果您不知道您的项目ID**，您可以使用 `gcloud` 获取您的项目ID。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

区域

您还可以更改“REGION”变量，该变量用于笔记本的其余部分。以下是Vertex AI支持的区域。我们建议您选择离您最近的区域。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太：`asia-east1`

您可能无法使用多区域存储桶进行Vertex AI训练。并非所有区域都支持所有Vertex AI服务。

了解更多关于[Vertex AI区域](https://cloud.google.com/vertex-ai/docs/general/locations)。

In [ ]:
REGION = "[your-region]"  # @param {type: "string"}

if REGION == "[your-region]":
    REGION = "us-central1"

时间戳

如果您正在参加一个实时教程会话，您可能会使用一个共享测试账户或项目。为了避免用户在创建的资源之间发生名称冲突，您为每个实例会话创建一个时间戳，并将时间戳附加到您在本教程中创建的资源名称上。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### 验证您的Google Cloud帐户

**如果您正在使用Vertex AI Workbench笔记本**，您的环境已经经过验证。请跳过此步骤。

**如果您正在使用Colab**，运行下面的单元格，并在提示时依照指示通过oAuth验证您的帐户。

**否则**，请按照以下步骤操作：

在Cloud控制台中，转到[创建服务帐户密钥](https://console.cloud.google.com/apis/credentials/serviceaccountkey)页面。

1. **单击创建服务帐户**。

2. 在**服务帐户名称**字段中输入名称，然后单击**创建**。

3. 在**将此服务帐户授权给项目**部分，单击角色下拉列表。在过滤框中键入"Vertex AI"，然后选择**Vertex AI管理员**。在过滤框中键入"Storage Object Admin"，然后选择**存储桶对象管理员**。

4. 单击创建。包含您的密钥的JSON文件将下载到您的本地环境。

5. 在以下单元格中将您的服务帐户密钥路径输入为GOOGLE_APPLICATION_CREDENTIALS变量，并运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Vertex AI Workbench, then don't execute this code
IS_COLAB = "google.colab" in sys.modules
if not os.path.exists("/opt/deeplearning/metadata/env_version") and not os.getenv(
    "DL_ANACONDA_HOME"
):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

创建一个云存储桶

无论您使用的是什么笔记本环境，都需要按照以下步骤进行操作。

当您初始化用于 Python 的 Vertex AI SDK 时，您需要指定一个云存储暂存桶。暂存桶是您的数据集和模型资源相关数据在会话之间保留的位置。

请在下方设置您的云存储桶的名称。存储桶的名称在所有谷歌云项目中必须是全局唯一的，包括您组织之外的项目。

In [ ]:
BUCKET_NAME = "[your-bucket-name]"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "aip-" + TIMESTAMP
    BUCKET_URI = f"gs://{BUCKET_NAME}"

只有当您的存储桶尚不存在时：运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_URI

最后，通过检查云存储桶的内容来验证访问权限。

In [ ]:
! gsutil ls -al $BUCKET_URI

### 设置变量

接下来，设置一些在教程中使用的变量。
### 导入库

In [ ]:
import google.cloud.aiplatform as aiplatform

### 初始化用于Python的Vertex AI SDK

为您的项目和相应的存储桶初始化用于Python的Vertex AI SDK。

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_URI)

设置硬件加速器

您可以为训练任务设置硬件加速器。

设置变量 `TRAIN_GPU/TRAIN_NGPU`，以使用支持 GPU 的容器镜像和分配给虚拟机实例的 GPU 数量。例如，要使用一个支持 GPU 的容器镜像，每个虚拟机实例分配 4 个 Nvidia Telsa K80 GPU，您可以指定：

(aiplatform.gapic.AcceleratorType.NVIDIA_TESLA_K80, 4)

否则指定 `(None, None)` 来使用一个在 CPU 上运行的容器镜像。

了解更多关于[您的区域支持的硬件加速器](https://cloud.google.com/vertex-ai/docs/general/locations#accelerators)。

In [ ]:
if os.getenv("IS_TESTING_TRAIN_GPU"):
    TRAIN_GPU, TRAIN_NGPU = (
        aiplatform.gapic.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_TRAIN_GPU")),
    )
else:
    TRAIN_GPU, TRAIN_NGPU = (None, None)

设置预构建的容器

为训练设置预先构建的Docker容器镜像。

有关最新列表，请参阅[用于训练的预构建容器](https://cloud.google.com/ai-platform-unified/docs/training/pre-built-containers)。

In [ ]:
if os.getenv("IS_TESTING_TF"):
    TF = os.getenv("IS_TESTING_TF")
else:
    TF = "2.5".replace(".", "-")

if TF[0] == "2":
    if TRAIN_GPU:
        TRAIN_VERSION = "tf-gpu.{}".format(TF)
    else:
        TRAIN_VERSION = "tf-cpu.{}".format(TF)
else:
    if TRAIN_GPU:
        TRAIN_VERSION = "tf-gpu.{}".format(TF)
    else:
        TRAIN_VERSION = "tf-cpu.{}".format(TF)


TRAIN_IMAGE = "{}-docker.pkg.dev/vertex-ai/training/{}:latest".format(
    REGION.split("-")[0], TRAIN_VERSION
)

print("Training:", TRAIN_IMAGE, TRAIN_GPU, TRAIN_NGPU)

#### 设置机器类型

接下来，设置用于训练的机器类型。

- 设置变量 `TRAIN_COMPUTE` 来配置用于训练的 VMs 的计算资源。
 - `机器类型`
     - `n1-standard`: 每个 vCPU 3.75GB 内存。
     - `n1-highmem`: 每个 vCPU 6.5GB 内存
     - `n1-highcpu`: 每个 vCPU 0.9GB 内存
 - `vCPUs`: 数量范围为 \[2, 4, 8, 16, 32, 64, 96 \]

*注意：以下类型不支持用于训练：*

 - `standard`: 2个 vCPUs
 - `highcpu`: 2, 4 和 8 个 vCPUs

*注意：您也可以使用 n2 和 e2 机器类型进行训练和部署，但它们不支持 GPU。*

In [ ]:
if os.getenv("IS_TESTING_TRAIN_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_TRAIN_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
TRAIN_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Train machine type", TRAIN_COMPUTE)

## `Vertex AI实验`简介

使用`Vertex AI实验`，您可以在实验/开发模型架构和模型训练时记录和跟踪以下内容：

- 记录模型架构的元参数。
- 记录训练的超参数。
- 记录评估指标。
- 创建数据集、模型和评估的工件系谱。
- 将一个或多个训练运行组合在一个实验下。
- 比较实验结果。

`Vertex AI实验`可以与以下开发流程集成：

- 在笔记本中进行本地开发
- 在`Vertex AI训练`中进行云开发
- 将开发操作化为`Vertex AI管道`

了解更多关于[实验](https://cloud.google.com/vertex-ai/docs/experiments/)的信息。

了解更多关于[Vertex AI机器学习元数据简介](https://cloud.google.com/vertex-ai/docs/ml-metadata/introduction)。

### 在笔记本中进行本地开发

您可以在本地开发中跟踪一个实验，例如在Vertex AI Workbench笔记本中，方法如下：

- 将实验的创建进行封装（开场白）。
- 在实验中每次训练运行时实例化一个运行。
- 在本地训练运行中，记录相关参数和结果。
- 创建与工件和实验数据的关系。
- 检索实验数据。

创建用于跟踪训练相关元数据的实验

首先，使用`init()`方法创建一个实验，然后在实验中初始化一个运行，使用`start_run()`。

- `aiplatform.init()` - 创建一个实验实例
- `aiplatform.start_run()` - 在实验中跟踪特定的运行。

In [ ]:
# Specify a name for the experiment
EXPERIMENT_NAME = "[your-experiment-name]"

if EXPERIMENT_NAME == "[your-experiment-name]":
    EXPERIMENT_NAME = "example-" + TIMESTAMP

In [ ]:
# Create experiment
aiplatform.init(experiment=EXPERIMENT_NAME)
aiplatform.start_run("run-1")

### 为实验记录参数

通常，一个实验与特定数据集和模型架构相关联。在一个实验中，您可能会进行多个训练运行，每个运行尝试不同的配置。例如：

- 数据输入，比如：
    - 数据集拆分
    - 数据集抽样和提升
- 元参数，比如：
    - 层的深度和宽度
- 超参数，比如：
    - 批量大小
    - 学习率

这些配置设置被称为参数，您可以使用`log_params()`方法将其存储为键值对。

In [ ]:
metaparams = {}
metaparams["units"] = 128
aiplatform.log_params(metaparams)

hyperparams = {}
hyperparams["epochs"] = 100
hyperparams["batch_size"] = 32
hyperparams["learning_rate"] = 0.01
aiplatform.log_params(hyperparams)

记录实验的指标

在实验中完成或终止运行时，您可以记录结果，用于比较不同运行的结果。例如：

- 评估指标
- 超参数搜索选择
- 模型训​​练时间
- 提前停止触发器

这些结果被称为指标，您可以使用`log_metrics()`方法将其存储为键-值对。

In [ ]:
metrics = {}
metrics["test_acc"] = 98.7
metrics["train_acc"] = 99.3
aiplatform.log_metrics(metrics)

获取实验结果

当您在实验中完成一次运行时，调用`end_run()`方法来完成该运行的日志记录。

接下来，您可以使用实验名称作为参数传递给`get_experiment_df()`方法来获取实验结果作为pandas数据框。

In [ ]:
aiplatform.end_run()

experiment_df = aiplatform.get_experiment_df()
experiment_df = experiment_df[experiment_df.experiment_name == EXPERIMENT_NAME]
experiment_df.T

开始在实验中进行后续运行

接下来，您为同一实验创建第二次运行。在这个例子中，您将`units`的元参数从126更改为256，并记录不同的度量结果。

In [ ]:
aiplatform.start_run("run-2")

metaparams = {}
metaparams["units"] = 256  # changed the value
aiplatform.log_params(metaparams)

hyperparams = {}
hyperparams["epochs"] = 100
hyperparams["batch_size"] = 32
hyperparams["learning_rate"] = 0.01
aiplatform.log_params(hyperparams)

metrics = {}
metrics["test_acc"] = 98.8  # value changed
metrics["train_acc"] = 99.5  # value changed
aiplatform.log_metrics(metrics)

比较同一实验中的运行

最后，您将实验名称作为参数传递给`get_experiment_df()`方法，以获取实验中所有运行的结果作为pandas dataframe。

In [ ]:
aiplatform.end_run()

experiment_df = aiplatform.get_experiment_df()
experiment_df = experiment_df[experiment_df.experiment_name == EXPERIMENT_NAME]
experiment_df.T

删除实验

接下来，您可以使用 `delete()` 方法删除实验。

In [ ]:
exp = aiplatform.Experiment(EXPERIMENT_NAME)
exp.delete()

### 在实验运行中创建工件血统

在这个例子中，您可以在实验运行中添加工件血统。首先，您需要创建一个实验，然后在实验中开始运行。

In [ ]:
# Create experiment
aiplatform.init(experiment=EXPERIMENT_NAME)
aiplatform.start_run("run-1")

#### 创建数据集和模型工件

接下来，您将在Vertex AI ML Metadata中创建合成工件，与实验中的运行关联为血统。您将创建：

- `dataset_artifact`：作为实验运行输入的数据集。
- `model_artifact`：作为实验运行输出的模型。

In [ ]:
DATASET_URI = "gs://example/dataset.csv"
MODEL_URI = "gs://example/saved_model.pb"

dataset_artifact = aiplatform.Artifact.create(
    schema_title="system.Dataset", display_name="example_dataset", uri=DATASET_URI
)

model_artifact = aiplatform.Artifact.create(
    schema_title="system.Model", display_name="example_modl", uri=MODEL_URI
)

创建工件谱系

接下来，为实验运行创建工件谱系，您可以使用方法 `start_execution()` 实例化一个执行。然后，使用方法 `assign_input_artifacts()` 附加输入工件，使用方法 `assign_output_artifacts()` 附加输出工件。

在此示例中，为查找实验的谱系，您可以向执行运行添加一个合成（元数据）条目 `lineage`，并将值设置为谱系的控制台URI，您可以从方法 `get_output_artifacts()` 和属性 `lineage_console_uri` 中获取该值。

In [ ]:
with aiplatform.start_execution(
    schema_title="system.ContainerExecution", display_name="example_training"
) as execution:
    execution.assign_input_artifacts([dataset_artifact])

    aiplatform.log_params({"units": 256})
    aiplatform.log_metrics({"acc": 96.8})

    execution.assign_output_artifacts([model_artifact])

    aiplatform.log_metrics(
        {"lineage": execution.get_output_artifacts()[0].lineage_console_uri}
    )

#### 获取实验结果

接下来，您可以将实验名称作为参数传递给方法`get_experiment_df()`来获取实验结果作为一个pandas dataframe。

在这个示例中，您将资源URI存储到了执行运行中作为度量值`lineage`。

In [ ]:
aiplatform.end_run()

experiment_df = aiplatform.get_experiment_df()
experiment_df = experiment_df[experiment_df.experiment_name == EXPERIMENT_NAME]
experiment_df.T

请将文物谱系进行可视化处理

接下来，请打开以下链接以查看文物谱系。

In [ ]:
print(
    "Open the following link:", execution.get_output_artifacts()[0].lineage_console_uri
)

请删除工件的谱系。

接下来，使用delete()方法来删除工件的谱系。

In [ ]:
try:
    dataset_artifact.delete()
except Exception as e:
    print(e)
try:
    model_artifact.delete()
except Exception as e:
    print(e)

删除实验

接下来，您可以使用`delete()`方法来删除实验。

In [ ]:
exp.delete()

### 在`Vertex AI Training`中的云开发

您可以使用`Vertex AI Training`在云开发中跟踪一个实验，方法如下：

在您的Python训练脚本中，重复与本地开发相同的步骤：

- 包装（引导）实验的创建。
- 在实验中的每次训练运行中实例化一个运行。
- 在本地训练运行中，记录相应的参数和结果。
- 创建与工件和实验数据的关系。
- 检索实验数据。

### 包布局

在开始训练之前，您将了解如何为自定义训练作业组装Python包。解压后，该包包含以下目录/文件布局。

- PKG-INFO
- README.md
- setup.cfg
- setup.py
- trainer
  - \_\_init\_\_.py
  - task.py

文件`setup.cfg`和`setup.py`是将包安装到Docker镜像的操作环境中的说明。

文件`trainer/task.py`是执行自定义训练作业的Python脚本。*请注意*，当我们在工作进程池规范中引用它时，我们将目录斜杠替换为点(`trainer.task`)并删除文件后缀(`.py`)。

### 包组装

在下面的单元格中，您将组装训练包。

In [ ]:
# Make folder for Python training script
! rm -rf custom
! mkdir custom

# Add package information
! touch custom/README.md

setup_cfg = "[egg_info]\n\ntag_build =\n\ntag_date = 0"
! echo "$setup_cfg" > custom/setup.cfg

setup_py = "import setuptools\n\nsetuptools.setup(\n\n    install_requires=[\n\n        'google-cloud-aiplatform',\n\n  ],\n\n    packages=setuptools.find_packages())"
! echo "$setup_py" > custom/setup.py

pkg_info = "Metadata-Version: 1.0\n\nName: Synethic Training Script for Experiments\n\nVersion: 0.0.0\n\nSummary: Demostration training script\n\nHome-page: www.google.com\n\nAuthor: Google\n\nAuthor-email: aferlitsch@google.com\n\nLicense: Public\n\nDescription: Demo\n\nPlatform: Vertex"
! echo "$pkg_info" > custom/PKG-INFO

# Make the training subfolder
! mkdir custom/trainer
! touch custom/trainer/__init__.py

#### 创建合成训练脚本

首先，您需要编写一个合成训练脚本。它实际上并不会训练模型，而是模仿模型的训练：

- 参数解析
  - `experiment`：实验的名称。
  - `run`：实验中的运行名称。
  - `epochs`：迭代次数。
  - `dataset-uri`：训练数据的云存储位置。
  - `model-dir`：保存训练后的模型文件的云存储位置。
- 训练函数
  - `get_data()`：
      - 获取训练数据。
      - 创建输入数据集工件。
      - 将数据集工件作为输入附加到执行上下文中。
  - `get_model()`：
      - 获取模型架构。
  - `train_model()`：
      - 训练模型。
  - `save_model()`：
      - 保存模型。
      - 创建输出模型工件。
      - 将模型工件作为输出附加到执行上下文中。
- 初始化实验（`init()`）并在实验内启动一个运行（`start_run()`）。
- 使用 `start_execution()` 包装训练过程。
- 将实验参数的谱系记录到日志中（`log_metrics({"lineage"...)`）。
- 结束实验运行（`end_run()`）。

In [ ]:
%%writefile custom/trainer/task.py

import argparse
import os

import google.cloud.aiplatform as aiplatform

parser = argparse.ArgumentParser()
# Args for experiment
parser.add_argument('--experiment', dest='experiment',
                    required=True, type=str,
                    help='Name of experiment')
parser.add_argument('--run', dest='run',
                    required=True, type=str,
                    help='Name of run within the experiment')

# Hyperparameters for experiment
parser.add_argument('--epochs', dest='epochs',
                    default=10, type=int,
                    help='Number of epochs.')

parser.add_argument('--dataset-uri', dest='dataset_uri',
                    required=True, type=str,
                    help='Location of the dataset')

parser.add_argument('--model-dir', dest='model_dir',
                    default=os.getenv("AIP_MODEL_DIR"), type=str,
                    help='Storage location for the model')
args = parser.parse_args()

def get_data(dataset_uri, execution):
    # get the training data
    
    dataset_artifact = aiplatform.Artifact.create(
        schema_title="system.Dataset", display_name="example_dataset", uri=dataset_uri
    )
    
    execution.assign_input_artifacts([dataset_artifact])

    return None

def get_model():
    # get or create the model architecture
    return None

def train_model(dataset, model, epochs):
    aiplatform.log_params({"epochs": epochs})
    # train the model
    return model

def save_model(model, model_dir, execution):
    # save the model
    
    model_artifact = aiplatform.Artifact.create(
        schema_title="system.Model", display_name="example_model", uri=model_dir
    )
    execution.assign_output_artifacts([model_artifact])

# Create a run within the experiment
aiplatform.init(experiment=args.experiment)
aiplatform.start_run(args.run)

with aiplatform.start_execution(
    schema_title="system.ContainerExecution", display_name="example_training"
) as execution:
    dataset = get_data(args.dataset_uri, execution)
    model = get_model()
    model = train_model(dataset, model, args.epochs)
    save_model(model, args.model_dir, execution)
    
    # Store the lineage link in the experiment
    aiplatform.log_metrics({"lineage": execution.get_output_artifacts()[0].lineage_console_uri})

aiplatform.end_run()

将培训脚本存储在您的云存储桶中

接下来，您将培训文件夹打包成一个压缩的tar球，然后存储在您的云存储桶中。

In [ ]:
! rm -f custom.tar custom.tar.gz
! tar cvf custom.tar custom
! gzip custom.tar
! gsutil cp custom.tar.gz $BUCKET_URI/trainer.tar.gz

创建自定义训练任务

使用`CustomTrainingJob`类创建自定义训练任务，具有以下参数：

- `display_name`：自定义训练任务的可读名称。
- `container_uri`：训练容器镜像。

- `python_package_gcs_uri`：Python训练包的位置，以tarball格式。
- `python_module_name`：Python包中训练脚本的相对路径。

*注意：*没有要求参数。您可以在Python包的`setup.py`脚本中指定任何要求。

In [ ]:
DISPLAY_NAME = "example_" + TIMESTAMP

job = aiplatform.CustomPythonPackageTrainingJob(
    display_name=DISPLAY_NAME,
    python_package_gcs_uri=f"{BUCKET_URI}/trainer.tar.gz",
    python_module_name="trainer.task",
    container_uri=TRAIN_IMAGE,
)

运行自定义训练任务

接下来，您可以通过调用方法`run()`运行自定义训练作业来开始训练作业，参数如下：

- `args`：传递给训练脚本的参数
    - `model_dir`：存储模型的云存储位置。
    - `dataset_uri`：数据集的云存储位置。
    - `epochs`：训练周期的数量（超参数）。
    - `experiment`：实验的名称。
    - `run`：实验中的运行名称。
- `replica_count`：VM实例的数量。
- `machine_type`：每个VM实例的机器类型。

In [ ]:
CMDARGS = [
    "--model-dir=" + BUCKET_URI,
    "--dataset-uri=gs://example/foo.csv",
    "--epochs=5",
    f"--experiment={EXPERIMENT_NAME}",
    "--run=run-1",
]

job.run(args=CMDARGS, replica_count=1, machine_type=TRAIN_COMPUTE, sync=True)

#### 获取实验结果

接下来，您可以将实验名称作为参数传递给`get_experiment_df()`方法，以便获取实验结果作为一个pandas的数据框。

在这个例子中，您将资源URI存储到血统中，作为执行运行的一个指标值`lineage`。

In [ ]:
experiment_df = aiplatform.get_experiment_df()
experiment_df = experiment_df[experiment_df.experiment_name == EXPERIMENT_NAME]
experiment_df.T

#### 可视化文物世系

接下来，打开下面的链接可视化文物世系。

In [ ]:
print("Open the following link", experiment_df["metric.lineage"][0])

#### 删除自定义训练任务

您可以使用`delete()`方法来删除您的自定义训练任务。

In [ ]:
job.delete()

删除实验

由于实验是在`Vertex AI Training`中创建的，要删除实验，您可以使用`list()`方法获取项目中的所有实验，然后根据实验名称进行筛选。

In [ ]:
experiments = aiplatform.Experiment.list()
for experiment in experiments:
    if experiment.name == EXPERIMENT_NAME:
        experiment.delete()

# 清理

要清理此项目中使用的所有Google Cloud资源，您可以删除用于本教程的[Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的各个资源。

In [ ]:
! rm -rf custom

delete_bucket = False

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -rf {BUCKET_URI}